In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import IsolationForest
import time
from tqdm import tqdm
import os

current_dir = os.getcwd()
print(current_dir)

# Load metadata CSV
metadata_path = "meta_data_for_all_active_systems_for_calculation_20250401(CSV).csv"
metadata_df = pd.read_csv(metadata_path, dtype={"STATION_ID": str})  # Ensure STATION_ID is a string

# Get the first 840 rows of metadata
filtered_metadata = metadata_df.head(840).copy()

# Extract STATION_IDs
site_ids = filtered_metadata["STATION_ID"].tolist()

# Initialize results DataFrame
df_results = pd.DataFrame()

# Start progress bar
start_time = time.time()
with tqdm(total=len(site_ids), desc="Processing Stations", unit="station") as pbar:
    for i, DivrtID in enumerate(site_ids, start=1):
        end_date = datetime.today().strftime("%Y-%m-%d")
        api_url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={DivrtID}&end_date={end_date}&f=json"

        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                df = pd.DataFrame(data["data"], columns=["date", "value"])
                df.rename(columns={"date": "Date", "value": "DISCHARGE"}, inplace=True)
                df["STATION_ID"] = DivrtID

                df["Date"] = pd.to_datetime(df["Date"])
                df["SEASON"] = np.where(
                    df["Date"].dt.month.between(4, 9) | ((df["Date"].dt.month == 10) & (df["Date"].dt.day == 1)),
                    "Irrigation Season", "Non-Irrigation Season"
                )

                df["DISCHARGE"] = pd.to_numeric(df["DISCHARGE"], errors="coerce")
                df["FLAG_NEGATIVE"] = df["DISCHARGE"] < 0
                df["FLAG_ZERO"] = df["DISCHARGE"] == 0

                df_nonzero = df[df["DISCHARGE"] > 0].copy()

                if not df_nonzero.empty:
                    Q1, Q3 = df_nonzero["DISCHARGE"].quantile([0.25, 0.75])
                    IQR = Q3 - Q1
                    discharge_95th_percentile = np.percentile(df_nonzero["DISCHARGE"].dropna(), 95)

                    df_nonzero["RATE_OF_CHANGE"] = df_nonzero["DISCHARGE"].diff().abs()
                    df = df.merge(df_nonzero[["Date", "RATE_OF_CHANGE"]], on="Date", how="left")

                    df_nonzero["FLAG_REPEATED"] = df_nonzero["DISCHARGE"].groupby(
                        (df_nonzero["DISCHARGE"] != df_nonzero["DISCHARGE"].shift()).cumsum()
                    ).transform("count") >= 4

                    model = IsolationForest(contamination=0.05, random_state=42)
                    df_nonzero["OUTLIER_IF"] = model.fit_predict(df_nonzero[["DISCHARGE"]])
                    df_nonzero["OUTLIER_IF"] = df_nonzero["OUTLIER_IF"] == -1

                    mean_discharge = df_nonzero["DISCHARGE"].mean()
                    df["PERCENT_DEV"] = ((df["DISCHARGE"] - mean_discharge).abs() / mean_discharge) * 100

                    threshold = 1000
                    df["FLAG_RSD"] = (df["PERCENT_DEV"] > threshold) & (df["DISCHARGE"] != 0)

                    df = df.merge(df_nonzero[["Date", "OUTLIER_IF", "FLAG_REPEATED"]], on="Date", how="left")
                else:
                    discharge_95th_percentile = 0
                    IQR = 0
                    Q1 = Q3 = 0
                    df["RATE_OF_CHANGE"] = np.nan
                    df["OUTLIER_IF"] = False
                    df["FLAG_REPEATED"] = False
                    df["PERCENT_DEV"] = np.nan
                    df["FLAG_RSD"] = False

                df["FLAG_Discharge"] = df["DISCHARGE"] > discharge_95th_percentile
                df["FLAG_IQR"] = (df["DISCHARGE"] < Q1 - 1.5 * IQR) | (df["DISCHARGE"] > Q3 + 1.5 * IQR)
                df["FLAG_RoC"] = df["RATE_OF_CHANGE"] > discharge_95th_percentile

                df["FLAGGED"] = df[
                    ["FLAG_NEGATIVE", "FLAG_ZERO", "FLAG_REPEATED", "FLAG_IQR", "OUTLIER_IF", "FLAG_Discharge", "FLAG_RoC", "FLAG_RSD"]
                ].any(axis=1)

                total_flagged = df["FLAGGED"].sum()

                station_summary = {
                    "STATION_ID": DivrtID,
                    "TOTAL_RECORDS": len(df),
                    "TOTAL_NEGATIVE": df["FLAG_NEGATIVE"].sum(),
                    "TOTAL_ZERO": df["FLAG_ZERO"].sum(),
                    "TOTAL_95th": df["FLAG_Discharge"].sum(),
                    "TOTAL_IQR": df["FLAG_IQR"].sum(),
                    "TOTAL_RoC": df["FLAG_RoC"].sum(),
                    "TOTAL_REPEATED": df["FLAG_REPEATED"].sum(),
                    "TOTAL_IF": df["OUTLIER_IF"].sum(),
                    "TOTAL_RSD": df["FLAG_RSD"].sum(),
                    "TOTAL_FLAGGED": total_flagged,
                    "NEGATIVE_RATIO": (df["FLAG_NEGATIVE"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "ZERO_RATIO": (df["FLAG_ZERO"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "DISCHARGE_RATIO": (df["FLAG_Discharge"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "IQR_RATIO": (df["FLAG_IQR"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "RoC_RATIO": (df["FLAG_RoC"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "REPEATED_RATIO": (df["FLAG_REPEATED"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "IF_RATIO": (df["OUTLIER_IF"].sum() / total_flagged) * 100 if total_flagged else 0,
                    "RSD_RATIO": (df["FLAG_RSD"].sum() / total_flagged) * 100 if total_flagged else 0,
                }

                # === Seasonal Summaries === #
                df_ir = df[df["SEASON"] == "Irrigation Season"]
                df_nir = df[df["SEASON"] == "Non-Irrigation Season"]

                total_ir_flagged = df_ir["FLAGGED"].sum()
                total_nir_flagged = df_nir["FLAGGED"].sum()

                irrigation_season_summary = {
                    "STATION_ID": DivrtID,
                    "TOTAL_IR_RECORDS": len(df_ir),
                    "TOTAL_IR_NEGATIVE": df_ir["FLAG_NEGATIVE"].sum(),
                    "TOTAL_IR_ZERO": df_ir["FLAG_ZERO"].sum(),
                    "TOTAL_IR_95th": df_ir["FLAG_Discharge"].sum(),
                    "TOTAL_IR_IQR": df_ir["FLAG_IQR"].sum(),
                    "TOTAL_IR_RoC": df_ir["FLAG_RoC"].sum(),
                    "TOTAL_IR_REPEATED": df_ir["FLAG_REPEATED"].sum(),
                    "TOTAL_IR_IF": df_ir["OUTLIER_IF"].sum(),
                    "TOTAL_IR_RSD": df_ir["FLAG_RSD"].sum(),
                    "TOTAL_IR_FLAGGED": total_ir_flagged,
                    "NEGATIVE_IR_RATIO": (df_ir["FLAG_NEGATIVE"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "ZERO_IR_RATIO": (df_ir["FLAG_ZERO"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "DISCHARGE_IR_RATIO": (df_ir["FLAG_Discharge"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "IQR_IR_RATIO": (df_ir["FLAG_IQR"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "RoC_IR_RATIO": (df_ir["FLAG_RoC"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "REPEATED_IR_RATIO": (df_ir["FLAG_REPEATED"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "IF_IR_RATIO": (df_ir["OUTLIER_IF"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                    "RSD_IR_RATIO": (df_ir["FLAG_RSD"].sum() / total_ir_flagged) * 100 if total_ir_flagged else 0,
                }

                non_irrigation_season_summary = {
                    "STATION_ID": DivrtID,
                    "TOTAL_NIR_RECORDS": len(df_nir),
                    "TOTAL_NIR_NEGATIVE": df_nir["FLAG_NEGATIVE"].sum(),
                    "TOTAL_NIR_ZERO": df_nir["FLAG_ZERO"].sum(),
                    "TOTAL_NIR_95th": df_nir["FLAG_Discharge"].sum(),
                    "TOTAL_NIR_IQR": df_nir["FLAG_IQR"].sum(),
                    "TOTAL_NIR_RoC": df_nir["FLAG_RoC"].sum(),
                    "TOTAL_NIR_REPEATED": df_nir["FLAG_REPEATED"].sum(),
                    "TOTAL_NIR_IF": df_nir["OUTLIER_IF"].sum(),
                    "TOTAL_NIR_RSD": df_nir["FLAG_RSD"].sum(),
                    "TOTAL_NIR_FLAGGED": total_nir_flagged,
                    "NEGATIVE_NIR_RATIO": (df_nir["FLAG_NEGATIVE"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "ZERO_NIR_RATIO": (df_nir["FLAG_ZERO"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "DISCHARGE_NIR_RATIO": (df_nir["FLAG_Discharge"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "IQR_NIR_RATIO": (df_nir["FLAG_IQR"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "RoC_NIR_RATIO": (df_nir["FLAG_RoC"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "REPEATED_NIR_RATIO": (df_nir["FLAG_REPEATED"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "IF_NIR_RATIO": (df_nir["OUTLIER_IF"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                    "RSD_NIR_RATIO": (df_nir["FLAG_RSD"].sum() / total_nir_flagged) * 100 if total_nir_flagged else 0,
                }

                # Combine all summaries into one dictionary per station
                combined_summary = {
                    **station_summary,
                    **{k: v for k, v in irrigation_season_summary.items() if k != "STATION_ID"},
                    **{k: v for k, v in non_irrigation_season_summary.items() if k != "STATION_ID"}
                }
                
                # Append as one row
                df_results = pd.concat([df_results, pd.DataFrame([combined_summary])], ignore_index=True)

        pbar.update(1)

# Merge and export
merged_df = pd.merge(filtered_metadata, df_results, on="STATION_ID", how="left")
output_filename = "merged_seasonal_flagging_dvrt_data_840_stations_results_20250409.csv"
merged_df.to_csv(output_filename, index=False)

print(f"Final merged data saved as {output_filename}")

print ("Done")


C:\Users\pbenko\Documents\20250213_distribution_data_python\data\QC_Scripts\criteria_protocol\Multiple_station_csv_file_20250401


Processing Stations: 100%|██████████| 839/839 [08:53<00:00,  1.57station/s]

Final merged data saved as merged_seasonal_flagging_dvrt_data_840_stations_results_20250409.csv
Done
